In [4]:
import requests
import subprocess
import time

DOCKER_CONTAINER_NAME = "dartboard-api-container"
DOCKER_IMAGE_NAME = "dartboard-api:latest"
API_URL = "http://localhost:8000"

def is_api_running() -> bool:
    """Check if the API is running by sending a request."""
    try:
        response = requests.get(API_URL)
        return response.status_code == 200
    except requests.ConnectionError:
        return False

def is_container_running() -> bool:
    """Check if the Docker container is running."""
    result = subprocess.run(
        ["docker", "ps", "--filter", f"name={DOCKER_CONTAINER_NAME}", "--format", "{{.Names}}"],
        capture_output=True,
        text=True
    )
    return DOCKER_CONTAINER_NAME in result.stdout.strip()

def start_container():
    """Start the Docker container if it is not running."""
    if is_container_running():
        print(f"Container '{DOCKER_CONTAINER_NAME}' is already running.")
        return

    print(f"Starting container '{DOCKER_CONTAINER_NAME}'...")
    subprocess.run(["docker", "run", "-d", "-p", "8000:8000", "--name", DOCKER_CONTAINER_NAME, DOCKER_IMAGE_NAME])
    
    # Wait for the API to become available
    for _ in range(10):
        time.sleep(2)
        if is_api_running():
            print("API is running!")
            return

    print("Failed to start API.")

def stop_container():
    """Stop the Docker container if it is running."""
    if is_container_running():
        print(f"Stopping container '{DOCKER_CONTAINER_NAME}'...")
        subprocess.run(["docker", "stop", DOCKER_CONTAINER_NAME])
        subprocess.run(["docker", "rm", DOCKER_CONTAINER_NAME])
    else:
        print(f"Container '{DOCKER_CONTAINER_NAME}' is not running.")




In [5]:

start_container()

# Check API response
if is_api_running():
    print("API is accessible at:", API_URL)
    response = requests.get(API_URL)
    print("Response:", response.text)
else:
    print("API is not responding.")



Container 'dartboard-api-container' is already running.
API is accessible at: http://localhost:8000
Response: {"message":"Welcome to the Dart API!"}


In [7]:
# Send a GET request to the endpoint
response = requests.get(f"{API_URL}/scores/throw", params={'radius':0.6, 'angle':90})

# Check the response
if response.status_code == 200:
    # Success: Print the response JSON
    print("Response JSON:", response.json())
else:
    # Error: Print the status code and error message
    print("Error:", response.status_code, response.text)

Response JSON: {'multiplier': 3, 'zone': 20, 'score': 60}


In [8]:
# Stop the container after 10 seconds (for demo purposes)
time.sleep(10)
stop_container()

Stopping container 'dartboard-api-container'...
dartboard-api-container
dartboard-api-container
